In [1]:
import pandas as pd
import numpy as np
import datetime
import csv

#%matplotlib inline

# Leo csv

In [2]:
test_final = pd.read_csv('../data/test_final_100k.csv')
test_final = test_final.drop('id',axis=1)

In [3]:
avisos_detalle = pd.read_csv('../data/datos_navent_fiuba/fiuba_6_avisos_detalle.csv')
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv'))
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_desde_15_abril/fiuba_6_avisos_detalle.csv'))
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv'))
avisos_detalle = avisos_detalle.reset_index().drop('index',axis=1)

In [4]:
postulantes_genero_y_edad = pd.read_csv('../data/datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv')
postulantes_genero_y_edad = postulantes_genero_y_edad.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_2_postulantes_genero_y_edad.csv'))
postulantes_genero_y_edad = postulantes_genero_y_edad.append(pd.read_csv('../data/fiuba_desde_15_abril/fiuba_2_postulantes_genero_y_edad.csv'))
postulantes_genero_y_edad = postulantes_genero_y_edad.reset_index().drop('index',axis=1)

In [5]:
postulaciones = pd.read_csv('../data/datos_navent_fiuba/fiuba_4_postulaciones.csv')
postulaciones = postulaciones.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_4_postulaciones.csv'))
postulaciones = postulaciones.reset_index().drop('index',axis=1)

In [6]:
vistas = pd.read_csv('../data/datos_navent_fiuba/fiuba_3_vistas.csv')
vistas = vistas.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_3_vistas.csv'))
vistas = vistas.append(pd.read_csv('../data/fiuba_desde_15_abril/fiuba_3_vistas.csv'))
vistas = vistas.reset_index().drop('index',axis=1)

# Pre-Procesamiento de datos

#### Guardo los postulantes y avisos del test_final_100k para luego quedarme con estos datos de los demas csv

In [7]:
postulantesDelTest = test_final.drop_duplicates(subset=['idpostulante'],keep='last')
avisosDelTest  = test_final.drop_duplicates(subset=['idaviso'],keep='last')

postulantesDelTest = postulantesDelTest.drop('idaviso',axis=1)
avisosDelTest  =  avisosDelTest.drop('idpostulante',axis=1)

#### Me quedo con los postulantes del test y elimino los duplicados del csv del genero y edad

In [8]:
postulantes_genero_y_edad = postulantes_genero_y_edad.merge(postulantesDelTest,on='idpostulante',how='inner')

postulantes_genero_y_edad = postulantes_genero_y_edad.drop_duplicates()
postulantes_genero_y_edad = postulantes_genero_y_edad.drop_duplicates(subset=['idpostulante'],keep='last')
postulantes_genero_y_edad = postulantes_genero_y_edad.reset_index().drop('index',axis=1)

#### Agrego la edad de los postulantes

In [9]:
postulantes_genero_y_edad['fechanacimiento'] = pd.to_datetime(postulantes_genero_y_edad['fechanacimiento'],errors='coerce')
postulantes_genero_y_edad['edad'] = datetime.datetime.now().year - postulantes_genero_y_edad['fechanacimiento'].dt.year

In [10]:
postulantes_genero_y_edad = postulantes_genero_y_edad.drop(['fechanacimiento'],axis=1)

#### Relleno los valores Nan de la edad, con la media

In [11]:
# En un principio los relleno con la media de los postulantes
postulantes_genero_y_edad.loc[postulantes_genero_y_edad['edad'].isnull(),'edad'] = int(postulantes_genero_y_edad['edad'].mean())
postulantes_genero_y_edad['edad'] = postulantes_genero_y_edad['edad'].astype('int32')

#### Reemplazo los valores invalidos de la columna sexo por NO_DECLARA

In [12]:
postulantes_genero_y_edad.loc[postulantes_genero_y_edad['sexo'] == '0.0','sexo'] = 'NO_DECLARA'

#### Me deshago de las columnas que no voy a usar y elimino duplicados del csv de avisos detalle

In [13]:
avisos_detalle = avisos_detalle.drop(['ciudad','mapacalle','nombre_zona','descripcion','titulo','tipo_de_trabajo','nivel_laboral','denominacion_empresa','idpais'],axis=1)

avisos_detalle = avisos_detalle.drop_duplicates(subset=['idaviso'],keep='last')
avisos_detalle = avisos_detalle.reset_index().drop('index',axis=1)

#### Elimino duplicados del csv de postulaciones y me quedo solo con los postulantes que aparecen en el test final

In [14]:
postulaciones = postulaciones.drop_duplicates(keep='last')
postulaciones = postulaciones.drop_duplicates(subset=['idaviso','idpostulante'],keep='last')
postulaciones = postulaciones.reset_index().drop('index',axis=1)

postulaciones = postulaciones.merge(postulantesDelTest,on='idpostulante',how='inner')

#### Elimino duplicados del csv de vistas y me quedo solo con los avisos y postulantes que aparecen en el test final

In [15]:
#Se me ocurrió que un postulante pudo ver un aviso mas de una vez.
#Creo  que con lo de abajo limpio todos pero no estoy seguro.
vistas = vistas.drop_duplicates(keep='last')
vistas = vistas.drop_duplicates(subset=['idAviso','idpostulante'],keep='last')
vistas = vistas.reset_index().drop('index',axis=1)

vistas = vistas.rename(columns={'idAviso': 'idaviso'})

In [16]:
vistas = vistas.merge(postulantesDelTest,on='idpostulante',how='inner')
vistas = vistas.merge(avisosDelTest,on='idaviso',how='inner')

# Funciones

In [17]:
# Recibe las series de idpostulante e idaviso
def generador_de_sample_serie(idaviso, idpostulante, n_sample, ciclos):
    idpostulante = pd.DataFrame(idpostulante)
    idaviso = pd.DataFrame(idaviso)
    idpostulante['tem'] = 1
    idaviso['tem'] = 1
    sample_append = pd.DataFrame(columns=[idpostulante.columns[0],'tem',idaviso.columns[0]])
    for i in range(0,ciclos):
        sample_idpostulante = idpostulante.sample(n=n_sample,random_state=i)
        sample_idaviso = idaviso.sample(n=n_sample,random_state=i)
        sample_merge = sample_idaviso.merge(sample_idpostulante)
        sample_append = sample_append.append(sample_merge)
    sample_append = sample_append.drop('tem',axis=1)
    sample_append = sample_append.reset_index(drop=True)
    return sample_append

In [18]:
def oneHotEncoding(df, columna):
    one_hot = pd.get_dummies(df[columna])
    df.drop(columna, axis=1, inplace=True)
    df = pd.concat([df, one_hot], axis=1)
    return df

In [19]:
def write_submission(test_data, prediction, file_output):
    
    archivo_entrada = open(test_data)
    entrada_csv = csv.reader(archivo_entrada)
    next(entrada_csv, None)  # skip the headers

    archivo_salida = open(file_output, 'w')
    submit_csv = csv.writer(archivo_salida)
    submit_csv.writerow(['id', 'sepostulo'])

    for reg1, reg2  in zip(entrada_csv, prediction):
        linea = [reg1[0], round(reg2, 2)]
        submit_csv.writerow(linea)
    archivo_salida.close()

In [20]:
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]

# Gente postulada y no postulada

## Tupla no postulada

#### La gente postulada sera la que aparece en el csv de postulaciones. Borro duplicados

In [21]:
gente_postulada = postulaciones
gente_postulada = gente_postulada.drop_duplicates(subset=['idpostulante'],keep='last')
gente_postulada = gente_postulada.reset_index().drop(['index','fechapostulacion'],axis=1)

In [22]:
gente_postulada.head()

,idaviso,idpostulante
0,1112094914,exNxxr
1,1112451206,1LXdVB
2,1112412656,6PWG4v
3,1112460141,eOk3oo
4,1112339122,5P9lqk


In [23]:
gente_postulada.shape

(46859, 2)

#### Hago samples que esten formados por la gente que se postulo alguna vez a algun aviso, junto con todos los avisos existentes

In [24]:
tupla_no_postulada = generador_de_sample_serie(avisos_detalle['idaviso'],gente_postulada['idpostulante'],100,134)

#### Luego hago un merge con las postulaciones. Los que tengan fechapostulacion en NaN son las tuplas no postuladas.

In [25]:
tupla_no_postulada = tupla_no_postulada.merge(postulaciones,how='left',on=['idaviso','idpostulante'])

In [26]:
tupla_no_postulada = tupla_no_postulada.fillna(0)
tupla_no_postulada = tupla_no_postulada.loc[tupla_no_postulada.fechapostulacion==0]
tupla_no_postulada = tupla_no_postulada.rename(columns={'fechapostulacion': 'y'})

In [27]:
cantidadDeNoPostulados = tupla_no_postulada.shape[0]
cantidadDeNoPostulados

1338559

In [28]:
tupla_no_postulada.head()

,idaviso,idpostulante,y
0,1112390041,66G584,0
1,1112390041,NRJlDx,0
2,1112390041,akjGwWJ,0
3,1112390041,aGbYA9,0
4,1112390041,Rz6V2QY,0


## Tupla postulada

In [29]:
tupla_postulada=postulaciones
tupla_postulada['y'] = 1

In [30]:
tupla_postulada.head()

,idaviso,idpostulante,fechapostulacion,y
0,1112305215,exNxxr,2018-02-06 13:30:25,1
1,1112286277,exNxxr,2018-02-06 13:31:30,1
2,1112275246,exNxxr,2018-02-06 13:31:48,1
3,1112265392,exNxxr,2018-02-06 13:32:21,1
4,1112315858,exNxxr,2018-02-06 13:35:03,1


In [31]:
tupla_postulada.shape

(1344469, 4)

# Tupla final para entrenar

In [32]:
tuplas = tupla_no_postulada.append(tupla_postulada.sample(n=cantidadDeNoPostulados,random_state=0))

In [33]:
tuplas.head()

,fechapostulacion,idaviso,idpostulante,y
0,NaN,1112390041,66G584,0
1,NaN,1112390041,NRJlDx,0
2,NaN,1112390041,akjGwWJ,0
3,NaN,1112390041,aGbYA9,0
4,NaN,1112390041,Rz6V2QY,0


In [34]:
tuplas.tail()

,fechapostulacion,idaviso,idpostulante,y
860171,2018-03-12 17:07:07,1112335159,vVjGva5,1
245649,2018-04-03 07:46:26,1112437070,vVjAWwQ,1
1315496,2018-01-28 23:08:26,1112277683,Z1MbGZ,1
684108,2018-01-31 16:11:59,1112297028,mzVAOAZ,1
480832,2018-03-19 14:47:51,1112410167,aExN6N,1


In [ ]:
tuplas = tuplas.drop('fechapostulacion',axis=1)

In [35]:
tuplas['y'].value_counts()

1    1338559
0    1338559
Name: y, dtype: int64

# Funciones feature

In [36]:
def feature_genero_edad(tupla_idaviso_idpostulante,postulantes_genero_y_edad):
    df_return = tupla_idaviso_idpostulante.merge(postulantes_genero_y_edad,on='idpostulante',how='left')
    return df_return

In [37]:
def feature_area_de_trabajo(tupla_a_predecir):
    df_return = tupla_a_predecir.merge(avisos_detalle,on='idaviso',how='left')
    return df_return

In [38]:
def feature_visto(tupla_a_predecir):
    df_return = tupla_a_predecir.merge(vistas,on=['idaviso','idpostulante'],how='left')
    return df_return

# Features en tuplas

## Feature Genero

In [39]:
tuplas = feature_genero_edad(tuplas,postulantes_genero_y_edad)

In [41]:
tuplas = oneHotEncoding(tuplas,'sexo')

## Feature visto

#### Agrego la columna 'visto' al csv de vistas que indica con un 1 si el aviso fue visto por cierto postulante

In [42]:
vistas['visto'] = 1
vistas = vistas.drop('timestamp',axis=1)

#### Se hace un merge de las tuplas con el csv de vistas, si la columna 'visto' queda en Nan significa que ese aviso no fue visto por ese postulante

In [43]:
tuplas=feature_visto(tuplas)

#### Los que quedaron en Nan se reemplazan por 0

In [44]:
tuplas.isnull().sum()

idaviso               0
idpostulante          0
y                     0
edad                  0
FEM                   0
MASC                  0
NO_DECLARA            0
visto           2289428
dtype: int64

In [45]:
tuplas = tuplas.fillna(0)

In [46]:
tuplas.visto.value_counts()

0.0    2289428
1.0     387690
Name: visto, dtype: int64

In [47]:
tuplas = tuplas.drop(['idaviso','idpostulante'],axis=1)

In [48]:
tuplas.head()

,y,edad,FEM,MASC,NO_DECLARA,visto
0,0,46,1,0,0,0.0
1,0,37,0,1,0,0.0
2,0,24,0,1,0,0.0
3,0,31,1,0,0,0.0
4,0,26,0,1,0,0.0


# Features en Test Final

## Feature Genero y Edad

In [49]:
test_final=feature_genero_edad(test_final,postulantes_genero_y_edad)

#### Los datos invalidos de la columna sexo se reemplazan por NO_DECLARA y los datos nulos de la edad se reemplazan por la media

In [50]:
# Salvo datos nulos
test_final.loc[test_final['sexo'].isnull(),'sexo'] = 'NO_DECLARA'
test_final.loc[test_final['edad'].isnull(),'edad'] = int(postulantes_genero_y_edad['edad'].mean())

In [51]:
test_final['edad'] = test_final['edad'].astype('int32')

In [52]:
test_final = oneHotEncoding(test_final,'sexo')

In [53]:
test_final.head()

,idaviso,idpostulante,edad,FEM,MASC,NO_DECLARA
0,739260,6M9ZQR,42,1,0,0
1,739260,6v1xdL,31,0,1,0
2,739260,ezRKm9,36,1,0,0
3,758580,1Q35ej,69,0,1,0
4,758580,EAN4J6,32,1,0,0


## Feature Visto

#### Se hace un merge de las tuplas con el csv de vistas, si la columna 'visto' queda en Nan significa que ese aviso no fue visto por ese postulante

In [54]:
test_final=feature_visto(test_final)

#### Los que quedaron en Nan se reemplazan por 0

In [55]:
test_final = test_final.fillna(0)

In [56]:
test_final.visto.value_counts()

0.0    65839
1.0    34161
Name: visto, dtype: int64

In [57]:
test_final = test_final.drop(['idaviso','idpostulante'],axis=1)

In [58]:
test_final.head()

,edad,FEM,MASC,NO_DECLARA,visto
0,42,1,0,0,0.0
1,31,0,1,0,0.0
2,36,1,0,0,0.0
3,69,0,1,0,0.0
4,32,1,0,0,0.0


# Prueba Local

## Divido el set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tuplas.drop(columns='y'),tuplas['y'],random_state=0,test_size=0.01)

## Random Forest Classifier

In [ ]:
#-----------------------------------------------------------------
# Modelo Random Forest Classifier
#-----------------------------------------------------------------
from sklearn.ensemble import RandomForestClassifier

In [ ]:
ModeloRFC = RandomForestClassifier(n_estimators=100,random_state=0, n_jobs=-1)

In [ ]:
ModeloRFC.fit(X_train, y_train)

In [ ]:
result = ModeloRFC.predict(X_test)

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,result)

In [ ]:
#-----------------------------------------------------------------
# Modelo GradientBoostingClassifier
#-----------------------------------------------------------------
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
modeloGBC=GradientBoostingClassifier(n_estimators=100)

modeloGBC.fit(X_train, y_train)

In [ ]:
result = modeloGBC.predict(X_test)

In [ ]:
roc_auc_score(y_test,result)

# Prueba con Test Final

## Separo datos para entrenar y para test

In [59]:
train_label = []
train_data = []

train_label = tuplas.iloc[:, 0]
train_data = tuplas.iloc[:, tuplas.columns != 'y']

## Algoritmos de machine learning

In [ ]:
#-----------------------------------------------------------------
# Modelo Random Forest Classifier
#-----------------------------------------------------------------
from sklearn.ensemble import RandomForestClassifier

In [ ]:
ModeloRFC = RandomForestClassifier(n_estimators=100,random_state=0, n_jobs=-1)

In [ ]:
ModeloRFC.fit(train_data, train_label)

In [ ]:
result = ModeloRFC.predict(test_final)

In [ ]:
result.size

In [ ]:
write_submission("../data/test_final_100k.csv", result, "../predicciones/RFC_Alternativa.csv")

In [ ]:
prueba = pd.read_csv('../predicciones/RFC_Alternativa.csv')

In [ ]:
prueba.sepostulo.value_counts()

In [ ]:
del ModeloRFC

In [60]:
#-----------------------------------------------------------------
# Modelo GradientBoostingClassifier
#-----------------------------------------------------------------
from sklearn.ensemble import GradientBoostingClassifier

In [61]:
modeloGBC=GradientBoostingClassifier(n_estimators=100)

In [62]:
modeloGBC.fit(train_data, train_label)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [63]:
result = modeloGBC.predict(test_final)

In [64]:
result.size

100000

In [65]:
write_submission("../data/test_final_100k.csv", result, "../predicciones/GBC_Alternativa_final.csv")

In [66]:
prueba = pd.read_csv('../predicciones/GBC_Alternativa_final.csv')

In [67]:
prueba.sepostulo.value_counts()

0.0    55528
1.0    44472
Name: sepostulo, dtype: int64